<a href="https://colab.research.google.com/github/guptaa98/Kaggle-Notebooks/blob/master/text_generation_pytorch_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import string


In [2]:
data = "i am implementing lstm"

#Length of the sequence
seq_len = len(data)

In [3]:
seq_len

22

In [4]:
import string
letters = string.ascii_lowercase+' #'
n_letters = len(letters)
print('Letter set is '+letters)

Letter set is abcdefghijklmnopqrstuvwxyz #


In [5]:
#This function takes a character and returns an encoded vector. The encoding is one hot

def ltt(ch):
    ans = torch.zeros(n_letters)
    ans[letters.find(ch)]=1
    return ans
    
print("Encoding of 'a' ",ltt('a'))
print("Encoding of 'b' ",ltt('b'))
print("Encoding of '#' ",ltt('#'))

Encoding of 'a'  tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Encoding of 'b'  tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Encoding of '#'  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])


In [6]:
#This is our neural network class. every Neural Network in pytorch extends nn.Module

class MyLSTM(nn.Module):
    def __init__(self , input_dim , hidden_dim):
        super(MyLSTM,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        #LSTM takes, input dimensions, hidden dimensions and num layers in case of stacked LSTMs (Default is 1)
        self.LSTM = nn.LSTM(input_dim,hidden_dim)
        
    #Input must be 3 dimensional (Sequence len, batch, input dimensions)
    #hc is a tuple which contains the vectors h (hidden/feedback) and c (cell state vector)

    def forward(self,inp,hc):
        #this gives output for each input and also (hidden and cell state vector)

        output , _ = self.LSTM(inp,hc)
        return output

In [7]:
#Dimensions of output of neural network is (seq_len, batch , hidden_dim). Since we want output dimensions to be
#the same as n_letters, hidden_dim = n_letters (output dimensions = hidden_dimensions)

hidden_dim = n_letters     

#Invoking model. Input dimensions = n_letters i.e 28. output dimensions = hidden_dimensions = 28

model = MyLSTM(n_letters,hidden_dim)

optimizer = torch.optim.Adam(params = model.parameters(),lr=0.01)

LOSS = torch.nn.CrossEntropyLoss()

In [8]:
letters

'abcdefghijklmnopqrstuvwxyz #'

In [59]:
#List to store targets
targets = []

#Iterate through all chars in the sequence, starting from second letter. Since output for 1st letter is the 2nd letter
#print(data[1:])
for x in data[1:]+'#':
    #Find the target index. For a, it is 0, For 'b' it is 1 etc..
    #print(x)
    print(letters.find(x))
    print(x)
    targets.append(letters.find(x))
#Convert into tensor
#print(targets)
targets = torch.tensor(targets)
print("tensor targets",targets)

26
 
0
a
12
m
26
 
8
i
12
m
15
p
11
l
4
e
12
m
4
e
13
n
19
t
8
i
13
n
6
g
26
 
11
l
18
s
19
t
12
m
27
#
tensor targets tensor([26,  0, 12, 26,  8, 12, 15, 11,  4, 12,  4, 13, 19,  8, 13,  6, 26, 11,
        18, 19, 12, 27])


In [58]:
targets.size()

torch.Size([22])

In [15]:
#List to store input
inp_ohe = []
#Iterate through all inputs in the sequence
for c in data:
    #Convert into one hot encoding
    inp_ohe.append(ltt(c))

#Convert list to tensor
inp = torch.cat(inp_ohe,dim=0)
#Reshape tensor into 3 dimensions (sequence length, batches = 1, dimensions = n_letters (28))
inp = inp.view(seq_len,1,n_letters)

In [16]:
print(inp[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [33]:
#Let's note down start time to track the training time
#import time
#start = time.time()

#Number of iterations
n_iters = 50

for itr in range(n_iters):
    #Zero the previous gradients
    model.zero_grad()
    optimizer.zero_grad()

    #Initialize h and c vectors
    h = torch.rand(hidden_dim).view(1,1,hidden_dim)
    c = torch.rand(hidden_dim).view(1,1,hidden_dim)
    #print("hidden state",h)
    #print("cell state", c)
    
    #Find the output
    output = model(inp,(h,c))

    #Reshape the output to 2 dimensions. This is done, so that we can compare with target and get loss
    
    output = output.view(seq_len,n_letters)
    print(output[0])
    print(targets)
    #print(output.shape)
    #print(targets.shape)
    
    #Find loss
    loss = LOSS(output,targets)

    #Print loss for every 10th iteration
    if itr%10==0:
        print(itr , ' ' , (loss) )
    
    #Back propagate the loss
    loss.backward()
    #Perform weight updation
    optimizer.step()
    
#print((time.time()-start))

tensor([ 1.6966e-02, -7.4384e-01, -7.4006e-01, -7.4656e-01, -1.1752e-03,
        -7.4279e-01,  4.1651e-03, -7.4850e-01, -3.9501e-02, -7.4635e-01,
        -7.4700e-01, -6.9569e-01,  2.6418e-02, -5.6942e-01, -7.4997e-01,
         3.7345e-04, -7.4243e-01, -7.4574e-01, -6.9763e-01, -7.0259e-01,
        -7.4396e-01, -7.3863e-01, -7.4426e-01, -7.4525e-01, -7.4038e-01,
        -7.4549e-01,  8.6792e-01, -6.8609e-01], grad_fn=<SelectBackward>)
tensor([26,  0, 12, 26,  8, 12, 15, 11,  4, 12,  4, 13, 19,  8, 13,  6, 26, 11,
        18, 19, 12, 27])
0   tensor(1.8241, grad_fn=<NllLossBackward>)
tensor([ 3.5500e-03, -7.0518e-01, -7.1269e-01, -7.0300e-01, -3.2660e-01,
        -7.0461e-01,  3.0092e-04, -6.9328e-01, -3.8919e-01, -7.1664e-01,
        -7.1445e-01, -7.4168e-01,  1.5991e-02, -7.4453e-01, -7.1592e-01,
         1.0067e-03, -6.9861e-01, -7.0760e-01, -6.1710e-01, -5.5403e-01,
        -7.1388e-01, -6.8221e-01, -7.0454e-01, -7.1516e-01, -7.0006e-01,
        -7.1067e-01,  7.9811e-01, -6.7680e-01

In [34]:
#This utility method predicts the next letter given the sequence   
def predict(s):

    #Get the vector for input
    inp = s

    #Initialize h and c vectors
    h = torch.rand(1,1,hidden_dim)
    c = torch.rand(1,1,hidden_dim)
    
    #Get the output
    out = model(inp,(h,c))
    
    #Find the corresponding letter from the output
    return letters[out[-1][0].topk(1)[1].detach().numpy().item()]

In [35]:
test = "i am imple"

In [37]:
#List to store input
inp_tohe = []
#Iterate through all inputs in the sequence
for c in test:
    #Convert into one hot encoding
    inp_tohe.append(ltt(c))

#Convert list to tensor
s = torch.cat(inp_tohe,dim=0)
#Reshape tensor into 3 dimensions (sequence length, batches = 1, dimensions = n_letters (28))
s = s.view(len(test),1,n_letters)

In [38]:
predict(s)

'm'

In [45]:
test = "i am "

In [46]:
#List to store input
inp_tohe = []
#Iterate through all inputs in the sequence
for c in test:
    #Convert into one hot encoding
    inp_tohe.append(ltt(c))

#Convert list to tensor
s = torch.cat(inp_tohe,dim=0)
#Reshape tensor into 3 dimensions (sequence length, batches = 1, dimensions = n_letters (28))
s = s.view(len(test),1,n_letters)

In [47]:
predict(s)

'i'